In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import os

from utils import util_data, util_preprocess


In [3]:
config = configparser.ConfigParser()
config.read("config.ini")


['config.ini']

In [4]:
ds_conf = config["GSMARENA_DS"]
# Download the training and test datasets
dh = util_data.DownloadHelper(
    ds_conf["url"], ds_conf["name"], ds_conf["mode"], quiet=False
)

dh.download()

File gsm.csv already exists. Skip download.


In [12]:
df = dh.read_csv(low_memory=False, sep=",")
preprocess = util_preprocess.GSMArenaPreprocess(df)
df = preprocess.preprocess()

Removing unwanted columns and duplicates...
Shape before removing unwanted columns and duplicates: (10679, 86)
Shape after removing unwanted columns and duplicates: (9919, 24)

Preprocessing misc_price...
Number of rows before extracting valid misc_price: 9919
Number of rows after removing null values in misc_price: 6246
Number of rows extracted: 136
Number of rows extracted: 5866
Number of rows extracted: 71
Number of rows extracted: 71
Number of rows extracted: 7
Number of rows extracted: 0
Number of rows without value: 95
Number of rows after extracting valid misc_price: 6151

Preprocessing launch_announced...
Number of rows before extracting valid launch_announced: 6151
Number of rows after removing null values in launch_announced: 6151
Number of rows extracted: 6142
Number of rows without a value: 9
Number of rows after extracting valid launch_announced: 6142

Preprocessing display_size...
Number of rows before extracting valid display_size: 6142
Number of rows after removing null

In [13]:
print(f"The dataframe has {df.shape[0]} rows and {df.shape[1]} columns.")

The dataframe has 2647 rows and 21 columns.


Filter conditions for recent mobile phones

- launch_announced >= 2014
- display_size in (4, 7.5)
- display_height and display_width > 512

In [14]:
filter_condition = {
    "launch_announced": lambda x: x >= 2014,
    "display_size": lambda x: x >= 4,
    "display_size": lambda x: x <= 7.5,
    "display_height": lambda x: x >= 512,
    "display_width": lambda x: x >= 512,
}
df = preprocess.filter_unwanted_rows(filter_condition)

Removing unwanted rows and duplicates...
Shape before removing unwanted rows and duplicates: (2647, 21)
Shape after removing unwanted rows and duplicates: (1973, 21)



In [15]:
print(f"The dataframe has {df.shape[0]} rows and {df.shape[1]} columns.")

The dataframe has 1973 rows and 21 columns.


In [16]:
out_path = os.path.join("data", "processed", ds_conf["name"])
df.to_csv(out_path, index=False)